In [2]:
#!/usr/bin/env python
# coding: utf-8

# In[213]:


import numpy as np
import pandas as pd


# In[214]:


df=pd.read_csv("F:\\gnr652\\assignment-rw\\FlightDelays.csv")
redf=pd.read_csv("F:\\gnr652\\assignment-rw\\FlightDelays.csv")


# In[215]:


class LogisticRegression:  
    def __init__(self, lr=0.01, num_iter=100000, fit_intercept=True, verbose=False):  
        self.lr = lr  
        self.num_iter = num_iter  
        self.fit_intercept = fit_intercept  
        self.verbose = verbose  
     
    def __add_intercept(self, X):  
        intercept = np.ones((X.shape[0], 1))  
        return np.concatenate((intercept, X), axis=1)  
     
    def __sigmoid(self, z):  
        return 1 / (1 + np.exp(-z))  
    def __loss(self, h, y):  
        return (-y * np.log(h) - (1 - y) * np.log(1 - h)).mean()  
     
    def fit(self, X, y):  
        if self.fit_intercept:  
            X = self.__add_intercept(X)  
         
        # weights initialization  
        self.theta = np.zeros(X.shape[1])  
         
        for i in range(self.num_iter):  
            z = np.dot(X, self.theta)  
            h = self.__sigmoid(z)  
            gradient = np.dot(X.T, (h - y)) / y.size  
            self.theta -= self.lr * gradient  
             
            z = np.dot(X, self.theta)  
            h = self.__sigmoid(z)  
            loss = self.__loss(h, y)  
                 
            if(self.verbose ==True and i % 10000 == 0):  
                print(f'loss: {loss} \t')  
     
    def predict_prob(self, X):  
        if self.fit_intercept:  
            X = self.__add_intercept(X)  
     
        return self.__sigmoid(np.dot(X, self.theta))  
     
    def predict(self, X):  
        return self.predict_prob(X).round()


# In[ ]:





# In[216]:


del df['FL_DATE']


# In[217]:


import math

for index, row in df.iterrows():
    df.loc[index, 'CRS_DEP_TIME'] = math.floor(row['CRS_DEP_TIME'] / 100)
    df.loc[index, 'DEP_TIME'] = math.floor(row['DEP_TIME'] / 100)


# In[218]:


#normalized_df=(df-df.min())/(df.max()-df.min())


# In[219]:


mx=df['DISTANCE'].max()

mi=df['DISTANCE'].min()
deno= mx-mi


# In[220]:


for index, row in df.iterrows():
    df.loc[index, 'DISTANCE'] = (df.loc[index, 'DISTANCE']-mi)/deno
    #df.loc[index, 'DEP_TIME'] = math.floor(row['DEP_TIME'] / 100)
#print(df['DISTANCE'])


# In[221]:


df['class'] = df['Flight Status'].apply(lambda x : 1 if x == "ontime" else 0)


# In[ ]:





# In[ ]:





# In[ ]:





# In[ ]:





# In[ ]:





# In[222]:


X=df.drop("class",1)


# In[223]:


Y=df['class']


# In[224]:


df=pd.get_dummies(df, columns=['DEST' ,'FL_NUM','ORIGIN','CARRIER','TAIL_NUM', 'CRS_DEP_TIME','DEP_TIME','DAY_WEEK' ,'DAY_OF_MONTH'])


# In[225]:


del df['Flight Status']


# In[226]:


X=df.drop("class",1)


# In[227]:


#list(X.columns)


# In[ ]:





# In[ ]:





# In[228]:


pd.set_option('display.max_rows',900)


# In[229]:


msk = np.random.rand(len(df)) < 0.6
#print(msk)


# In[230]:


X_train = X[msk]

X_test = X[~msk]

Y_train = Y[msk]

Y_test = Y[~msk]
Y_train.reset_index(inplace=True, drop=True)
Y_test.reset_index(inplace=True, drop=True)
#print(Y_test)


# In[231]:


#df.isnull().any().any()


# In[232]:


model = LogisticRegression(lr=0.01, num_iter=3000)  
get_ipython().magic(u'time model.fit(X_train, Y_train)')


# In[233]:


preds_test = model.predict(X_test)
preds_train = model.predict(X_train)


# In[234]:


#print(preds)


# In[235]:


#print(model.theta)  


# In[236]:


#print(preds)


# In[237]:


#type(preds)


# In[238]:


q = pd.DataFrame(np.around(preds_test))
q['train']=pd.DataFrame(np.around(preds_train))


# In[239]:


q['class']=Y_test
q['ctr']=Y_train
#print(q)


# In[240]:


#print('Test Accuracy:',q.loc[q[0]==q['class']].shape[0] / q.shape[0] * 100)
#print('Train Accuracy:',q.loc[q['train']==q['ctr']].shape[0] / q.shape[0] * 100)


# In[241]:


from sklearn import metrics
#print('Test Precision Score: ',metrics.precision_score(Y_test, preds_test))
#print('Test Recall Score: ',metrics.recall_score(Y_test, preds_test))
#print('Test f1 Score: ',metrics.f1_score(Y_test, preds_test))


# In[242]:





# In[252]:


# Threshold for removing correlated variables
threshold = 0.9
# Absolute value correlation matrix
corr_matrix = redf.corr().abs()
corr_matrix.head()


# Upper triangle of correlations
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
print(upper)

to_drop = [column for column in upper.columns if any(upper[column] > threshold)]

print('There are %d columns to remove.' % (len(to_drop)))
print(to_drop)


# In[245]:


from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2


# In[246]:


bestfeatures = SelectKBest(score_func=chi2, k=10)
fit = bestfeatures.fit(X_train,Y_train)
dfscores = pd.DataFrame(fit.scores_)
dfcolumns = pd.DataFrame(X_train.columns)
#concat two dataframes for better visualization


# In[249]:


featureScores = pd.concat([dfcolumns,dfscores],axis=1)
featureScores.columns = ['Specs','Score']  #naming the dataframe columns
pd.set_option('display.max_rows',900)
print(featureScores.nlargest(300,'Score'))  #print 10 best features



# In[251]:


X1=(featureScores.nlargest(300,'Score'))['Specs']
X1.reset_index(inplace=True, drop=True)
#print(X1)
#list(X1)


# In[ ]:


Wall time: 13 s
              CRS_DEP_TIME  DEP_TIME  DISTANCE    FL_NUM   Weather  DAY_WEEK  \
CRS_DEP_TIME           NaN  0.983523  0.062368  0.086920  0.008266  0.051766   
DEP_TIME               NaN       NaN  0.057680  0.105660  0.019001  0.051868   
DISTANCE               NaN       NaN       NaN  0.421937  0.033104  0.020926   
FL_NUM                 NaN       NaN       NaN       NaN  0.042076  0.018604   
Weather                NaN       NaN       NaN       NaN       NaN  0.123636   
DAY_WEEK               NaN       NaN       NaN       NaN       NaN       NaN   
DAY_OF_MONTH           NaN       NaN       NaN       NaN       NaN       NaN   

              DAY_OF_MONTH  
CRS_DEP_TIME      0.002324  
DEP_TIME          0.000132  
DISTANCE          0.010121  
FL_NUM            0.009268  
Weather           0.144034  
DAY_WEEK          0.015914  
DAY_OF_MONTH           NaN  
There are 1 columns to remove.
['DEP_TIME']
               Specs      Score
1            Weather  71.004016
703